In [2]:
# # Debugging ONLY! autoreloads when scripts update
# %load_ext autoreload
# %autoreload 2

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import contextily as ctx

import os

from src import preprocessing, config, plotting, utils

# Obtain raw data

In [2]:
# Get 2020 Decennial Census data
decennial2020_dp_raw = preprocessing.fetch_2020_demographic_profile()

GET request status: 200


In [ ]:
np.array(decennial2020_dp_raw).shape

(2328, 646)

In [3]:
# Get 2023 ACS 5yr data
acs5yr2023_raw = preprocessing.fetch_2023_acs_5yr_select()
np.array(acs5yr2023_raw).shape

GET request status: 200


(2328, 18)

# Perform initial cleaning

In [3]:
# Clean NYC census tract shapefiles
gdf = preprocessing.clean_tracts(config.TRACTS_RAW, config.TRACTS_CLEAN, config.AREAWATER)

gdf.sample(n=5)

,BOROUGH,TRACT,AREA,LAT,LONG,geometry
GEOID,,,,,,
36085002900,Staten Island,29,526903,40.619658,-74.082741,"POLYGON ((-74.08783 40.61532, -74.0877 40.6155..."
36061019000,Manhattan,190,103795,40.801961,-73.947620,"POLYGON ((-73.94968 40.80202, -73.94919 40.802..."
36047047700,Brooklyn,477,217806,40.718218,-73.942137,"POLYGON ((-73.94679 40.71882, -73.94574 40.719..."
36047026100,Brooklyn,261,199415,40.692399,-73.945714,"POLYGON ((-73.94891 40.6939, -73.94608 40.6942..."
36081097205,Queens,972.05,176096,40.596225,-73.785514,"POLYGON ((-73.78921 40.59518, -73.78918 40.595..."


In [13]:
decennial2020_dp_clean = preprocessing.clean_2020_demographic_profile()
print(decennial2020_dp_clean.shape)
decennial2020_dp_clean.sample(n=5)

(2222, 153)


,pop,pop_under5,pop_5to9,pop_10to14,pop_15to19,pop_20to24,pop_25to29,pop_30to34,pop_35to39,pop_40to44,pop_45to49,pop_50to54,pop_55to59,pop_60to64,pop_65to69,pop_70to74,pop_75to79,pop_80to84,pop_85plus,pop_16plus,pop_18plus,pop_21plus,pop_62plus,pop_65plus,pop_male,pop_male_under5,pop_male_5to9,pop_male_10to14,pop_male_15to19,pop_male_20to24,pop_male_25to29,pop_male_30to34,pop_male_35to39,pop_male_40to44,pop_male_45to49,pop_male_50to54,pop_male_55to59,pop_male_60to64,pop_male_65to69,pop_male_70to74,pop_male_75to79,pop_male_80to84,pop_male_85plus,pop_male_16plus,pop_male_18plus,pop_male_21plus,pop_male_62plus,pop_male_65plus,pop_female,pop_female_under5,pop_female_5to9,pop_female_10to14,pop_female_15to19,pop_female_20to24,pop_female_25to29,pop_female_30to34,pop_female_35to39,pop_female_40to44,pop_female_45to49,pop_female_50to54,pop_female_55to59,pop_female_60to64,pop_female_65to69,pop_female_70to74,pop_female_75to79,pop_female_80to84,pop_female_85plus,pop_female_16plus,pop_female_18plus,pop_female_21plus,pop_female_62plus,pop_female_65plus,medianage,medianage_male,medianage_female,pop_onerace,pop_white,pop_black,pop_nativeam,pop_asian,pop_hipacific,pop_other,pop_multi,racetally,racetally_white,racetally_black,racetally_nativeam,racetally_asian,racetally_hipacific,racetally_other,pop_hispanic,pop_nonhispanic,pop_hispanic_white,pop_hispanic_black,pop_hispanic_nativeam,pop_hispanic_asian,pop_hispanic_hipacific,pop_hispanic_other,pop_hispanic_multi,pop_nonhisp_white,pop_nonhisp_black,pop_nonhisp_nativeam,pop_nonhisp_asian,pop_nonhisp_hipacific,pop_nonhisp_other,pop_nonhisp_multi,pop_household,pop_household_householder,pop_household_spouse_oppsex,pop_household_spouse_samesex,pop_household_unmpartner_oppsex,pop_household_unmpartner_samesex,pop_household_child,pop_household_child_under18,pop_household_grandchild,pop_household_grandchild_under18,pop_household_otherrelatives,pop_household_nonrelatives,pop_gpquarters,pop_gpquarters_institutionalized,pop_gpquarters_institutionalized_male,pop_gpquarters_institutionalized_female,pop_gpquarters_noninst,pop_gpquarters_noninst_male,pop_gpquarters_noninst_female,households,households_married,households_married_withchild,households_cohabit,households_cohabit_withchild,households_singlemale,households_singlemale_alone,households_singlemale_alone_65plus,households_singlemale_withchild,households_singlefemale,households_singlefemale_alone,households_singlefemale_alone_65plus,households_singlefemale_withchild,households_withunder18,households_with65plus,housing,housing_occupied,housing_vacant,housing_vacant_forrent,housing_vacant_rentedunocc,housing_vacant_forsale,housing_vacant_soldunocc,housing_vacant_seasonal,housing_vacant_other,vacancyrate_homeowner,vacancyrate_rental,housing_owneroccupied,housing_renteroccupied
GEOID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36081040502,2175,115,153,179,145,194,158,195,198,180,150,152,114,84,45,49,25,16,23,1689,1632,1550,206,158,1219,62,89,96,77,118,101,112,104,102,97,79,64,49,25,17,12,2,13,952,922,874,99,69,956,53,64,83,68,76,57,83,94,78,53,73,50,35,20,32,13,14,10,737,710,676,107,89,34.1,32.9,34.8,1911,168,80,85,102,0,1476,264,2465,370,120,136,109,1,1729,1943,232,133,25,85,2,0,1451,247,35,55,0,100,0,25,17,2175,560,231,6,49,2,695,420,51,43,315,266,0,0,0,0,0,0,0,566,242,144,50,31,139,49,7,16,135,32,10,45,286,119,586,566,20,13,0,0,0,0,7,0.0,2.6,77,489
36047115200,3496,237,251,308,274,325,319,260,242,222,199,208,187,156,100,90,55,29,34,2655,2528,2370,385,308,1615,128,111,142,153,171,158,122,109,85,89,84,80,65,47,29,21,13,8,1216,1137,1045,147,118,1881,109,140,166,121,154,161,138,133,137,110,124,107,91,53,61,34,16,26,1439,1391,1325,238,190,30.6,28.7,33.6,3018,196,1992,34,147,10,639,478,3979,369,2304,59,184,19,1044,1202,2294,116,167,23,7,2,609,278,80,1825,11,140,8,30,200,3482,1079,286,2,48,3,1247,737,171,135,445,201,14,0,0,0,14,9,5,1086,296,132,49,31,225,121

In [14]:
# Remove NYC tracts that are not in the DP or are unpopulated
gdf = preprocessing.remove_lowpop_tracts()
gdf.shape

(2222, 6)

In [15]:
# Clean 2023 ACS 5-year data
acs5yr2023_clean = preprocessing.clean_2023_acs_5yr_select()

print(acs5yr2023_clean.shape)
acs5yr2023_clean.sample(n=10)

(2222, 7)


,pct_sameresidence1yr,pct_englishonlyhome,pct_commute_transit,pct_commute_walk,medianhouseholdincome,pct_incomebelowpoverty,medianhomevalue
GEOID,,,,,,,
36047028700,87.0,58.9,54.9,11.0,79350.0,33.8,941100.0
36081014201,95.6,24.6,38.8,4.4,63409.0,7.7,966500.0
36061008603,87.9,72.4,14.4,28.9,189926.0,4.5,1204100.0
36081087100,89.4,21.4,33.5,17.0,47662.0,23.9,984600.0
36005046000,90.8,70.0,36.3,0.4,31768.0,27.7,474500.0
36047008200,89.7,31.8,61.6,7.3,43142.0,30.5,1058500.0
36081094500,95.3,36.0,21.9,2.6,79700.0,10.9,1032400.0
36081114100,93.8,54.7,26.4,0.9,126429.0,3.3,1024200.0
36081005500,88.7,35.1,51.4,11.8,95833.0,9.3,1203100.0


In [16]:
acs5yr2023_clean.isna().sum()

pct_sameresidence1yr        0
pct_englishonlyhome         0
pct_commute_transit         3
pct_commute_walk            3
medianhouseholdincome      33
pct_incomebelowpoverty      3
medianhomevalue           286
dtype: int64